In [3]:
import websocket
import json
from urllib.parse import quote

In [2]:
ws = websocket.WebSocket()
website_id='google.com'
ws.connect(f'wss://plankton-app-5rfza.ondigitalocean.app/websites?scraping={website_id}')
ws.recv()

NameError: name 'websocket' is not defined

In [ ]:
ws.close()

In [5]:
ws = websocket.WebSocket()
page_id = 'https://betway.com/es/sports/evt/10682299","websiteId'
website_id='betway.com'
# uri = 'wss://plankton-app-5rfza.ondigitalocean.app'
uri='ws://localhost:8000'
ws.connect(f'{uri}/games/{website_id}?game_id={page_id}')
ws.recv()

'{"status": "error", "message": "Game does not exist"}'

In [36]:
from core.models import Website, Game, Bet, delete, save, exists
g = Game(**{"id": "https://betway.com/es/sports/evt/10682300", "websiteId": "betway.com", "scraping": True})
g.key()

r.json().get('games:betway.com:/es/sports/evt/10682300')

In [7]:
from core.models import r
from redis.commands.search.query import Query, NumericFilter
from  redis.commands.search import aggregation
from redis.commands.search import reducers

In [123]:
r.ft('idxGames').search(Query('@scraping:{false}').paging(0, 5))

Result{163 total, docs: [Document {'id': 'games:betway.com:/es/sports/evt/10683641', 'payload': None, 'json': '{"id":"https://betway.com/es/sports/evt/10683641","websiteId":"betway.com","urlSource":"https://betway.com/es/sports/cpn/soccer/90","sport":"futbol","fullName":"Atherton Collieries vs FC United of Manchester","firstTeam":"Atherton Collieries","secoundTeam":"FC United of Manchester","scraping":false}'}, Document {'id': 'games:betway.com:/es/sports/evt/10652376', 'payload': None, 'json': '{"id":"https://betway.com/es/sports/evt/10652376","websiteId":"betway.com","urlSource":"https://betway.com/es/sports/cpn/soccer/90","sport":"futbol","fullName":"Famalicão vs Tondela","firstTeam":"Famalicão","secoundTeam":"Tondela","scraping":false}'}, Document {'id': 'games:betway.com:/es/sports/evt/10683207', 'payload': None, 'json': '{"id":"https://betway.com/es/sports/evt/10683207","websiteId":"betway.com","urlSource":"https://betway.com/es/sports/cpn/soccer/90","sport":"futbol","fullName":"

In [130]:
q = '*Marrue*'
query = Query(f"{q}").with_scores().paging(0, 30).language('spanish')
query2 = Query(q.replace(' ', '%')).with_scores().paging(0, 30).language('spanish')
r.ft('idxGames').search(query)

ResponseError: Syntax error at offset 1 near Marrue

In [95]:
r.ft('idxGames').sugget('games:', 'US')

[]

In [45]:
from core.webdriver import init_driver
from selenium.webdriver.common.by import By
from core.models import Bet
from time import sleep
driver = init_driver()
url='https://betway.com/es/sports/evt/10682245'
driver.get(url)

In [48]:
sleep(10)
status_code = driver.find_element(By.CSS_SELECTOR, 'meta[http-equiv="status"]').get_attribute('content')
print(status_code == '404')

True


In [21]:
def get_bids(page_url):
    for elm in driver.find_elements(By.CSS_SELECTOR, '.collapsablePanel'):
        collapsed_elm = elm.find_element(By.CSS_SELECTOR, '.collapsableContent')
        if 'empty' in collapsed_elm.get_attribute('class'):
            try:
                elm.click()
            except:
                pass
        try:
            header_elm = elm.find_element(By.CSS_SELECTOR, '.titleText>.title')
            cashout_elm = elm.find_element(By.CSS_SELECTOR, '.cashOutMarketIndicatorContainer')
        except:
            continue    
        try:
            labels_elms = elm.find_elements(By.CSS_SELECTOR, '.outcomeHeader, .outcomeItemHeader')
            odds_elms = elm.find_elements(By.CSS_SELECTOR, '.oddsDisplay')
        except:
            continue
        
        for labels_elm, odds_elm in zip(labels_elms, odds_elms):
            try:
                group = header_elm.text.strip()
                name = labels_elm.text.strip()
                bet = Bet(
                    id=f'{group}-{name}'.replace(' ', '-').lower(),
                    websiteId='betway.com',
                    gameId=page_url,
                    group=group, 
                    name=name, 
                    bet=float(odds_elm.text.replace(',', '.'))
                )
            except Exception as e:
                print('Error', e)

In [22]:
get_bids(url)

1-x-2-usc-paloma
1-x-2-empate
1-x-2-harksheide
ambos-equipos-marcan-sí
ambos-equipos-marcan-no
combinación-ganador-del-partido-y-los-dos-equipos-marcan-usc-paloma-gana-y-ambos-equipos-marcan
combinación-ganador-del-partido-y-los-dos-equipos-marcan-empate-y-ambos-equipos-marcan
Error could not convert string to float: ''
ganador-del-partido-y-ambos-equipos-marcan-sí/no-usc-paloma-gana-y-sí
ganador-del-partido-y-ambos-equipos-marcan-sí/no-empate-y-sí
ganador-del-partido-y-ambos-equipos-marcan-sí/no-harksheide-gana-y-sí
ganador-del-partido-y-ambos-equipos-marcan-sí/no-usc-paloma-gana-y-no
ganador-del-partido-y-ambos-equipos-marcan-sí/no-empate-y-no
ganador-del-partido-y-ambos-equipos-marcan-sí/no-harksheide-gana-y-no
doble-oportunidad-usc-paloma-o-empate
doble-oportunidad-usc-paloma-o-harksheide
doble-oportunidad-empate-o-harksheide
doble-oportunidad-y-ambos-equipos-marcan-usc-paloma-o-empate
doble-oportunidad-y-ambos-equipos-marcan-usc-paloma-o-harksheide
doble-oportunidad-y-ambos-equipo